In [73]:
def compress_fen(fen):
    """
    Compresses a FEN representation by replacing consecutive '1's with their count.
    
    Args:
        fen (str): The input FEN string to be compressed.
    
    Returns:
        str: The compressed FEN string.
    """
    compressed_fen = []
    
    for row in fen.split('/'):
        count = 0
        new_row = ""
        
        for char in row:
            if char == '1':
                count += 1
            else:
                if count > 0:
                    new_row += str(count)
                    count = 0
                new_row += char
                
        if count > 0:
            new_row += str(count)
        
        compressed_fen.append(new_row)
    
    return '/'.join(compressed_fen)


def expand_fen(fen):
    """
    Expands a compressed FEN representation by replacing counts with '1's.
    
    Args:
        fen (str): The input compressed FEN string to be expanded.
    
    Returns:
        str: The expanded FEN string.
    """
    expanded_fen = []
    
    for row in fen.split('/'):
        new_row = ""
        
        for char in row:
            if char.isdigit():
                new_row += '1' * int(char)  # Repeat '1' based on the number
            else:
                new_row += char
        
        expanded_fen.append(new_row)
    
    return '/'.join(expanded_fen)




In [74]:
# Example Usage
original_fen = "11111111/11111P11/11111111/11111111/11111111/11111111/11111111/1111111r"
compressed = compress_fen(original_fen)
expanded = expand_fen(compressed)

print("Original FEN: ", original_fen)
print("Compressed FEN: ", compressed)
print("Expanded FEN: ", expanded)

Original FEN:  11111111/11111P11/11111111/11111111/11111111/11111111/11111111/1111111r
Compressed FEN:  8/5P2/8/8/8/8/8/7r
Expanded FEN:  11111111/11111P11/11111111/11111111/11111111/11111111/11111111/1111111r


In [75]:
def determine_chess_move(past_fen, current_fen):
    past_fen_list = past_fen.split("/")
    past_fen_list.reverse()
    current_fen_list = current_fen.split("/")
    current_fen_list.reverse()
    start = ""
    start_key = True
    start_pos = []
    finish = ""
    finish_key = True
    promote = ""
    for i in range (8):
        for j in range(8):
            if past_fen_list[i][j]!=current_fen_list[i][j] :
                if(current_fen_list[i][j] == '1')and(start_key):
                    start = chr(97 + j) + str(i+1)
                    start_pos = [i,j]
                    start_key = False
                elif finish_key:
                    finish = chr(97 + j) + str(i+1)
                    # Check for promotion if a pawn has reached the last rank
                    if i == 7 and past_fen_list[start_pos[0]][start_pos[1]] == 'P' :    #white pawn promotion
                        promote = current_fen_list[i][j].lower()
                    if i == 0 and past_fen_list[start_pos[0]][start_pos[1]] == 'p' :    #black pawn promotion
                        promote = current_fen_list[i][j].lower()
                    finish_key = False
                else:
                    print("past and current fen are seperated by multiple moves")
                    return '' #false value
    if start_key or finish_key:
        print("past and current fen are seperated by half a move or identical")
        return '' #false value
    
    return start + finish + promote

In [1]:
def is_one_move(past_fen, current_fen):
    past_fen_list = past_fen.split("/")
    past_fen_list.reverse()
    current_fen_list = current_fen.split("/")
    current_fen_list.reverse()
    
    white_pieces = ['P','N','R','B','Q','K']
    black_pieces = ['p','n','r','b','q','k']
    start_key = True
    finish_key = True
    for i in range (8):
        for j in range(8):
            #check if there is a change in the square and it is not pieces of the same color being mismatched (this function ignores miss detection of pieces of the same color)
            if past_fen_list[i][j]!=current_fen_list[i][j] and not ((past_fen_list[i][j] in white_pieces and current_fen_list[i][j] in white_pieces) or (past_fen_list[i][j] in black_pieces and current_fen_list[i][j] in black_pieces)):
                if current_fen_list[i][j] == '1' and start_key:
                    start_key = False
                elif finish_key:
                    finish_key = False
                else:
                    print("past and current fen are seperated by multiple moves")
                    return False #false value
    if start_key or finish_key:
        print("past and current fen are seperated by half a move or identical")
        return False #false value
    
    return True

In [76]:
import chess

def is_move_valid(past_fen, move):
    past_fen = compress_fen(past_fen)
    # Initialize the board with the previous FEN position
    board = chess.Board(past_fen)
    # Parse the move in UCI format
    try:
        chess_move = chess.Move.from_uci(move)
    except ValueError:
        print("Invalid move format.")
        return False
    # Check if the move is valid in the current position
    if chess_move in board.legal_moves:
        return True
    else:
        return False


In [77]:
# White pawn at f7 promotes to a queen on f8
past_fen = "11111111/11111P11/11111111/11111111/11111111/11111111/11111111/11111111"
current_fen = "11111Q11/11111111/11111111/11111111/11111111/11111111/11111111/11111111"
# Expected output: 'f7f8q'
determine_chess_move(past_fen, current_fen)


'f7f8q'

In [78]:
# Test cases

past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
move = "e2e4"  # White pawn moves from e2 to e4

print(is_move_valid(past_fen, move))  # Should print True if the move is valid


True


In [ ]:
def print_board_from_fen(fen):
    board = chess.Board(fen)
    print(board)


In [18]:
def fix_queen_king_issue(past_fen,current_fen):
    past_fen_list = [list(row) for row in past_fen.split("/")]
    current_fen_list = [list(row) for row in current_fen.split("/")]
    updated_fen_list = [list(row) for row in current_fen.split("/")]
    
    #Here we assume that the past fen is correct and the current fen is always right for any detections other than queen and king
    for i in range (8):
        for j in range(8):
            # 1. checks if king was confused with a queen
            # a. static
            if(past_fen_list[i][j] == 'k') and (current_fen_list[i][j] in ['q','n','r','b','p']):
                updated_fen_list[i][j] = 'k'
            # b. king made a move 
            if(current_fen_list[i][j] == '1') and (past_fen_list[i][j] == 'k'): #the king made a move
                #get all the neighbouring positions
                positions = [[x,y] for x in range (max(i-1,0),min(i+1,7)+1) for y in range (max(j-1,0),min(j+1,7)+1) if not (x==i and y==j)]
                for pos in positions:
                    if current_fen_list[pos[0]][pos[1]] == 'q' and past_fen_list[pos[0]][pos[1]] in['Q','B','P','R','N','1']:
                        updated_fen_list[pos[0]][pos[1]] = 'k'
                        break
            #same logic for white king 
            if(past_fen_list[i][j] == 'K') and (current_fen_list[i][j] in ['Q','N','R','B','P']):
                updated_fen_list[i][j] = 'K'            
            # b. king made a move 
            if(current_fen_list[i][j] == '1') and (past_fen_list[i][j] == 'K'): #the king made a move
                #get all the neighbouring positions
                positions = [[x,y] for x in range (max(i-1,0),min(i+1,7)+1) for y in range (max(j-1,0),min(j+1,7)+1) if not (x==i and y==j)]
                for pos in positions:
                    if current_fen_list[pos[0]][pos[1]] == 'Q' and past_fen_list[pos[0]][pos[1]] in['q','b','p','r','n','1']:
                        updated_fen_list[pos[0]][pos[1]] = 'K'
                        break
            #2. check if queen was confused with king
            # a. static
            if past_fen_list[i][j] in 'q' and current_fen_list[i][j] in ['k','n','r','b','p']:
                updated_fen_list[i][j] = 'q'
                
            # b. dynamic (queen made a move)
            if past_fen_list[i][j] == 'q' and current_fen_list[i][j] == '1':
                #we find all possible positions that he queen could take
                position = [[x,j] for x in range(8) if x!=i] + [[i,y] for y in range(8) if y!=j] + [[i+x,j+x] for x in range(-7,8) if 0<=i + x<=7 and 0<=j + x<=7 and x!=0 ] + [[i-x,j+x] for x in range(-7,8) if 0<=i - x<=7 and 0<=j + x<=7 and x!=0 ]
                
                for pos in position:
                    if current_fen_list[pos[0]][pos[1]] == 'k' and past_fen_list[pos[0]][pos[1]] not in ['k','b','p','r','n']:
                        updated_fen_list[pos[0]][pos[1]] = 'q'
                        break
            #check for other color
            # a. static
            if past_fen_list[i][j] in 'Q' and current_fen_list[i][j] in ['K','N','R','B','P']:
                updated_fen_list[i][j] = 'Q'
                
            # b. dynamic (queen made a move)
            if past_fen_list[i][j] == 'Q' and current_fen_list[i][j] == '1':
                #we find all possible positions that he queen could take
                position = [[x,j] for x in range(8) if x!=i] + [[i,y] for y in range(8) if y!=j] + [[i+x,j+x] for x in range(-7,8) if 0<=i + x<=7 and 0<=j + x<=7 and x!=0 ] + [[i-x,j+x] for x in range(-7,8) if 0<=i - x<=7 and 0<=j + x<=7 and x!=0 ]
                
                for pos in position:
                    if current_fen_list[pos[0]][pos[1]] == 'K' and past_fen_list[pos[0]][pos[1]] not in ['K','B','P','R','N']:
                        updated_fen_list[pos[0]][pos[1]] = 'Q'
                        break
                
    return '/'.join([''.join(row) for row in updated_fen_list])
                

In [32]:
def fix_fen(past_fen,current_fen):
    past_fen_list = [list(row) for row in past_fen.split("/")]
    current_fen_list = [list(row) for row in current_fen.split("/")]
    updated_fen_list = [list(row) for row in current_fen.split("/")]
    
    white_pieces = ['P','N','R','B','Q','K']
    black_pieces = ['p','n','r','b','q','k']
    
    for i in range (8):
        for j in range(8):
            #static checking 
            if past_fen_list[i][j]!=current_fen_list[i][j]:
                #Fix static miss predictions
                if (past_fen_list[i][j] in black_pieces and current_fen_list[i][j] in black_pieces) or (past_fen_list[i][j] in white_pieces and current_fen_list[i][j] in white_pieces):
                    updated_fen_list[i][j] = past_fen_list[i][j]
                #Fix dynamic miss predictions
            
                if(current_fen_list[i][j] == '1'):  #a move happened
                    #check  boxes based on move patterns of the chess pieces
                    positions = []
                    piece = past_fen_list[i][j]
                    if piece == 'p':
                        positions = [[i + 1,j + x] for x in range(-1,2) if 0 <= j + x <= 7 and i + 1 <= 7]
                    if piece == 'P':
                        positions = [[i - 1,j + x] for x in range(-1,2) if 0 <= j + x <= 7 and 0 <= i - 1]
                    elif piece in ['r', 'R']:
                        positions = [[x,j] for x in range(8) if x!=i] + [[i,y] for y in range(8) if y!=j]
                    elif piece in ['b', 'B']:
                        positions = [[i+x,j+x] for x in range(-7,8) if 0<=i + x<=7 and 0<=j + x<=7 and x!=0 ] + [[i-x,j+x] for x in range(-7,8) if 0<=i - x<=7 and 0<=j + x<=7 and x!=0 ]
                        print(positions)
                    elif piece in ['q', 'Q']:
                        positions = [[x,j] for x in range(8) if x!=i] + [[i,y] for y in range(8) if y!=j] + [[i+x,j+x] for x in range(-7,8) if 0<=i + x<=7 and 0<=j + x<=7 and x!=0 ] + [[i-x,j+x] for x in range(-7,8) if 0<=i - x<=7 and 0<=j + x<=7 and x!=0 ]
                    elif piece in ['k', 'K']:
                        positions = [[x,y] for x in range (max(i-1,0),min(i+1,7)+1) for y in range (max(j-1,0),min(j+1,7)+1) if not (x==i and y==j)]
                    else: #knignt :)
                        positions = [
                            [i + dx, j + dy]
                            for dx, dy in [
                                (2, 1), (2, -1), (-2, 1), (-2, -1),
                                (1, 2), (1, -2), (-1, 2), (-1, -2)
                            ]
                            if 0 <= i + dx <= 7 and 0 <= j + dy <= 7
                        ]
                    for pos in positions:
                        if (piece in white_pieces and current_fen_list[pos[0]][pos[1]] in white_pieces and past_fen_list[pos[0]][pos[1]] not in white_pieces) or (piece in black_pieces and current_fen_list[pos[0]][pos[1]] in black_pieces and past_fen_list[pos[0]][pos[1]] not in black_pieces):
                            updated_fen_list[pos[0]][pos[1]] = piece
                            
    return '/'.join([''.join(row) for row in updated_fen_list])

[[1, 0], [2, 1], [3, 2], [4, 3], [5, 4], [7, 6], [7, 4], [5, 6], [4, 7]]
11111111/11111111/11111111/11111111/11111k11/111111b1/11111111/11111111


In [28]:
def test_fix_queen_king_issue():
    # Test case 1: Static king misidentified as queen
    past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    current_fen = "rnbqqbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    expected = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    result = fix_queen_king_issue(past_fen, current_fen)
    print("Test 1:", "PASS" if result == expected else "FAIL")

    # Test case 2: King moves and misidentified as queen
    past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    current_fen = "rnbq1bnr/pppppppp/11111111/11111111/11111111/11111k11/PPPPPPPP/RNBQKBNR"
    expected = "rnbq1bnr/pppppppp/11111111/11111111/11111111/11111k11/PPPPPPPP/RNBQKBNR"
    result = fix_queen_king_issue(past_fen, current_fen)
    print("Test 2:", "PASS" if result == expected else "FAIL")

    # Test case 3: Static queen misidentified as king
    past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    current_fen = "rnbkkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    expected = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    result = fix_queen_king_issue(past_fen, current_fen)
    print("Test 3:", "PASS" if result == expected else "FAIL")

    # Test case 4: Queen moves diagonally and misidentified as king
    past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    current_fen = "rnb1kbnr/pppppppp/1k111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    expected = "rnb1kbnr/pppppppp/1q111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    result = fix_queen_king_issue(past_fen, current_fen)
    print("Test 4:", "PASS" if result == expected else "FAIL")

    # Test case 5: Queen moves horizontally and misidentified as king
    past_fen = "rnbqkbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQKBNR"
    current_fen = "rnb1kbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBKkBNR"
    expected = "rnb1kbnr/pppppppp/11111111/11111111/11111111/11111111/PPPPPPPP/RNBQkBNR"
    result = fix_queen_king_issue(past_fen, current_fen)
    print("Test 5:", "PASS" if result == expected else "FAIL")

In [29]:
test_fix_queen_king_issue()

Test 1: PASS
Test 2: PASS
Test 3: PASS
Test 4: PASS
Test 5: PASS
